In [1]:
%%capture
import numpy as np
import pandas as pd
!pip install openpyxl
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import os

In [2]:
actual_path = os.getcwd()
os.chdir('/home/onyxia/work/material_science/Spetral_clustering')
%run rbf.ipynb
os.chdir(actual_path)

In [3]:
max_calculated = pd.read_csv('max_calculated.csv')
max_elemental = pd.read_csv('max_elemental.csv')
list_mxene = pd.read_excel('synthesized-MXenes-MAX.xlsx',sheet_name=0)
list_failed = pd.read_excel('synthesized-MXenes-MAX.xlsx', sheet_name=2)
n_samples = max_elemental.shape[0]
synth_list = pd.unique(list_mxene['MXene'])[:-1]
to_drop = list(range(167,173))
mx_ene_df = list_mxene.drop(labels = to_drop, axis='index')
mx_ene_df = mx_ene_df.drop(['Unnamed: 9','Unnamed: 12','Notes','status','Reference method'],axis=1)
max_elemental['class'] = np.zeros(max_elemental.shape[0])
parents = mx_ene_df['Parent material'].unique()
banned_words = ['+','Mxene','topochemical','reaction', 'or',
               'synthesis','MXene','direct']
complete_parents = []
for i in range(len(parents)):
    inter = parents[i].split()
    for word in range(len(inter)):
        if inter[word] not in banned_words:
            complete_parents.append(inter[word])


for i in range(max_elemental.shape[0]):
    if max_elemental.loc[i,'compound_formula'] in complete_parents:
        max_elemental.loc[i,'class'] = 1

max_elemental = max_elemental.set_index('compound_formula',drop=True)
max_elemental = max_elemental.drop(['M_element', 'A_element', 'X_element'],axis=1)
max_calculated = max_calculated.set_index('prettyformula',drop=True)
whole_data = max_elemental.merge(max_calculated,how='inner',
                                 left_index=True,right_index=True)
whole_data = whole_data.drop(['PU_label','year'],axis=1)
M_elements = pd.get_dummies(whole_data['M'],prefix='M',dtype=float)
A_elements = pd.get_dummies(whole_data['A'],prefix='A',dtype=float)
X_elements = pd.get_dummies(whole_data['X'],prefix='X',dtype=float)
whole_data = whole_data.drop(['M','A','X'],axis=1)
x_group = pd.get_dummies(whole_data['X_X_group'],prefix='x_g',dtype=float)
a_group = pd.get_dummies(whole_data['A_A_group'],prefix='a_g',dtype=float)
m_group = pd.get_dummies(whole_data['M_M_group'],prefix='m_g',dtype=float)
whole_data = whole_data.drop(['X_X_group','A_A_group','M_M_group'],axis=1)
whole_data = pd.concat([whole_data,M_elements,A_elements,X_elements,x_group,
                       a_group,m_group],axis=1)

test_tree = DecisionTreeClassifier().fit(X=whole_data.drop(['class'],axis=1),
                                                              y=whole_data['class'])

imp_feat = test_tree.feature_importances_
names_feat = test_tree.feature_names_in_

imp_feat = imp_feat.reshape(-1,1)
names_feat = names_feat.reshape(-1,1)
test_df = pd.DataFrame(np.hstack((names_feat,imp_feat)))
test_df.columns = ['names_feat','imp_feat']
test_df = test_df.set_index('names_feat',drop=True)
test_df = test_df[test_df['imp_feat'] > 0]

diff_z = list(test_df.index)


number_of_atoms = np.zeros(n_samples)
compteur = 0
for element in whole_data.index:
    inter = []
    for cara in element:
        if cara in list(str(1234567890)):
            inter.append(cara)
    if len(inter) == 1:
        number_of_atoms[compteur] = int(inter[0]) + 2
    elif len(inter) == 2:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + 1
    elif len(inter) == 3:
        number_of_atoms[compteur] = int(inter[0]) + int(inter[1]) + int(inter[2])
    compteur += 1

columns_name = whole_data.drop(['class'],axis=1).columns.copy()
normalized = whole_data.drop(['class'],axis=1).to_numpy()/number_of_atoms.reshape(-1,1)

data_norm = pd.DataFrame(normalized)
data_norm.columns = columns_name
data_norm['compound_name'] = whole_data.index
data_norm = data_norm.set_index('compound_name',drop=True)

data_norm = data_norm.filter(items=list(diff_z),axis=1)
data_norm['class'] = whole_data['class'].copy()

retained_features = list(test_df.index)

for feat in diff_z:
    if len(feat) > 5:
        retained_features.remove(feat)

list_dummies = []

for i in retained_features:
    if 'M_' in i:
        list_dummies.append(i)
    elif 'A_' in i:
        list_dummies.append(i)
    elif 'X_' in i:
        list_dummies.append(i)

for col in list_dummies:
    for row in data_norm.index:
        if data_norm.loc[row,col] != 0:
            data_norm.loc[row,col] = 1

for i in data_norm.index:
    if data_norm.loc[i,'class'] == 0:
        data_norm.loc[i,'class'] = -1

In [4]:
very_positive_instances = data_norm[data_norm['class'] == 1].copy()

In [5]:
very_positive_instances['pred'] = np.zeros(15)

In [6]:
list_of_positive_part = list(very_positive_instances.index)

In [7]:
list_wo_class = list(data_norm.columns)
list_wo_class.remove('class')
df_whole_results = data_norm.drop(list_wo_class,axis=1)
t=15
count_elem = 0
n_positives = very_positive_instances.shape[0]-1
for part_element in list_of_positive_part:
    data = data_norm.drop(part_element,axis=0).copy()
    df_results = data.drop(list(data.columns),axis=1)
    positive_instances = data[data['class'] == 1]
    index_of_positive = list(positive_instances.index)
    unlabelled_instances = data.drop(labels=index_of_positive,axis=0)
    avg_element = np.zeros(t)
    for i in range(t):
        u_t = unlabelled_instances.sample(n=n_positives)
        training_set = pd.concat([positive_instances,u_t])
        training_index = list(training_set.index)
        clf = SVC().fit(X=training_set.drop(['class'],axis=1).to_numpy(),y=training_set['class'].to_numpy())
        interm_df = data.drop(labels=training_index,axis=0)
        interm_df['predictions'] = clf.decision_function(X=interm_df.drop(['class'],axis=1).to_numpy())
        df_for_merge = pd.DataFrame(interm_df['predictions'])
        df_for_merge.columns = [f'score_{i},{count_elem}']
        df_results = df_results.merge(df_for_merge, how='left', left_index=True, right_index=True)
        avg_element[i] = clf.decision_function(X=data_norm.drop(['class'],axis=1).loc[part_element,:].to_numpy().reshape(1,-1))[0]
    very_positive_instances.loc[part_element,'pred'] = np.mean(avg_element)
    df_whole_results = df_whole_results.merge(df_results,how='left',left_index=True,right_index=True)
    count_elem += 1

In [12]:
avg_results = df_whole_results.drop(['class'],axis=1).mean(axis=1)

In [13]:
very_positive_instances

,A_A_atomic_mass,dMM,dMA,dAA,rMX,rMA,dEc,volume,density,dH,M_Ti,M_V,A_Sn,A_Zn,class,pred
compound_name,,,,,,,,,,,,,,,,
Ti2AlC,6.745385,0.573418,0.572010,1.717437,0.527455,0.723328,1585.313992,27.986961,0.999209,-4.433290,1.0,0.0,0.0,0.0,1.0,0.536256
Ti2SC,8.016250,0.559185,0.424882,1.408950,0.540368,0.627973,1705.123542,25.046962,1.158582,-14.156324,1.0,0.0,0.0,0.0,1.0,0.609703
Ti2ZnC,16.352250,0.577213,0.571775,1.720762,0.527335,0.722298,1396.775752,27.866565,1.289542,6.033368,1.0,0.0,0.0,1.0,1.0,0.299133
V2AlC,6.745385,0.558340,0.541435,1.641212,0.504640,0.685475,1507.684761,24.113670,1.212639,-15.625497,0.0,1.0,0.0,0.0,1.0,0.510070
Nb2AlC,6.745385,0.623830,0.559453,1.742738,0.548930,0.719043,1702.981879,29.562210,1.578442,-4.223056,0.0,0.0,0.0,0.0,1.0,0.489108
Ta2AlC,6.745385,0.629637,0.557933,1.745500,0.546693,0.714878,1884.426031,28.988913,2.870454,-2.501170,0.0,0.0,0.0,0.0,1.0,0.738509
Ti3AlC2,4.496923,0.395053,0.382243,1.554592,0.356595,0.483758,1128.862198,25.565338,0.702228,-1.419006,1.0,0.0,0.0,0.0,1.0,-0.090953
Ti3SiC2,4.680917,0.399018,0.339795,1.477625,0.357025,0.450538,1168.330449,24.192273,0.746292,-1.351975,1.0,0.0,0.0,0.0,1.0,-0.054481
Ti3ZnC2,10.901500,0.395818,0.380173,1.551983,0.356583,0.481942,1046.059042,25.471238,0.843894,-1.186989,1.0,0.0,0.0,1.0,1.0,-0.294174


In [15]:
very_positive_instances['pred_bis'] = np.sign(very_positive_instances['pred'])

In [17]:
for_recall = very_positive_instances['pred_bis'].to_numpy()

In [18]:
true_positives = 0
for i in range(15):
    if for_recall[i] == 1:
        true_positives += 1

In [19]:
recall = true_positives/15

In [22]:
df_whole_results = df_whole_results.drop(['class'],axis=1)

In [23]:
df_avg = df_whole_results.mean(axis=1)

In [27]:
df_avg = pd.DataFrame(df_avg)
df_avg.columns = ['avg']
df_avg['decision'] = np.sign(df_avg['avg'])

In [34]:
predicted_positives = df_avg.groupby('decision').count().loc[1.,'avg']

In [36]:
proba_pos_pred = predicted_positives/data_norm.shape[0]

In [37]:
weird_metric = (recall**2)/proba_pos_pred

In [38]:
weird_metric

np.float64(0.5395545819642206)

In [39]:
recall

0.4666666666666667